Для парсинга метаданных используется Spotify API. 
1. Предварительно нужно создать аккаунт Spotify и создать приложение в Spotify Developer Dashboard: https://developer.spotify.com/dashboard/applications. В качестве redirect URI можно прописать адрес вроде http://localhost:8888/callback. Полученные после создания приложения client_id и сlient_secret понадобятся для получения access token. 
2. После создания приложения нужно перейти по этому адресу, подставив свои значения YOUR_CLIENT_ID и YOUR_REDIRECT_URI https://accounts.spotify.com/authorize?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=YOUR_REDIRECT_URI&scope=user-library-read, после чего понадобится авторизоваться в аккаунте Spotify и далее скопировать код из http://localhost:8888/callback?code=AUTHORIZATION_CODE. Код нужен для получения access token. Access token действителен в течение часа с момента получения, по истечении этого времени токен нужно обновить.
 

In [ ]:
# получаем токен доступа используя AUTHORIZATION_CODE
auth_code = 'AQBVfznV2l17b951nfr0GaCw11BJSvVjgVsTPfY07TPIOeby5FJGxFOV6rFXpAVCVHpT4eOAnKmCL6bfiqFBBxlnxd-rWSK6c320l1IUqfDJERKu3oBV1qmdwXfekLuhFyBU509UmWb6XeC36IVzy_xVzQWhrmLISa5jBeJbgN7GKTm5zKN1w2ozQr3b'  # Replace with the code from the redirect URI

token_data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri,
}

# кодируем client credentials используя base64
credentials = f'{client_id}:{client_secret}'.encode('utf-8')
b64_credentials = b64encode(credentials).decode('utf-8')

headers = {
    'Authorization': f'Basic {b64_credentials}',
}

response = requests.post(token_url, data=token_data, headers=headers)
token_info = response.json()

# The access token is in token_info['access_token']
access_token = token_info['access_token']

# You can use this access token to make requests to the Spotify Web API.


In [40]:
# тестируем на одном треке

import requests

# ISRC code тестового трека
isrc_code = 'USFI86900049'

# Define the URL to fetch track information using the Spotify API's search endpoint
url = f'https://api.spotify.com/v1/search'

# Set up the headers with the access token
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Define the parameters for the search request
params = {
    'q': f'isrc:{isrc_code}',  # Use the ISRC code for the search
    'type': 'track'  # Limit the search to tracks
}

# Make the API request to search for the track
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    # Successfully received search results
    search_results = response.json()

    # Extract information about the first track in the search results
    if 'tracks' in search_results and 'items' in search_results['tracks'] and search_results['tracks']['items']:
        track_data = search_results['tracks']['items'][0]

        # Extract artist and release date
        artist = ', '.join([artist['name'] for artist in track_data['artists']])
        release_date = track_data['album']['release_date']

        print(f'Artist: {artist}')
        print(f'Release Date: {release_date}')
    else:
        print('Track not found in search results.')
else:
    print(f'Error: {response.status_code}')


Artist: Creedence Clearwater Revival
Release Date: 1969-01-05


In [16]:
# код для парсинга информации об исполнителе и дате релиза для треков с текстами
import pandas as pd
from tqdm import tqdm  # Import tqdm

# тут подставить путь до датасета с isrc кодами
new_features = pd.read_csv('path_to_df')

# Define a function to retrieve artist and release date for an ISRC code
def fetch_artist_release_date(isrc):
    # Define the URL to fetch track information using the Spotify API's search endpoint
    url = f'https://api.spotify.com/v1/search'
    
    # Set up the headers with the access token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Define the parameters for the search request
    params = {
        'q': f'isrc:{isrc}',  # Use the ISRC code for the search
        'type': 'track'  # Limit the search to tracks
    }

    # Make the API request to search for the track
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        # Successfully received search results
        search_results = response.json()

        # Extract information about the first track in the search results
        if 'tracks' in search_results and 'items' in search_results['tracks'] and search_results['tracks']['items']:
            track_data = search_results['tracks']['items'][0]

            # Extract artist and release date
            artist = ', '.join([artist['name'] for artist in track_data['artists']])
            release_date = track_data['album']['release_date']

            return artist, release_date
        else:
            return None, None
    else:
        return None, None

# Initialize tqdm with the total number of ISRC codes
total_isrcs = len(new_features)
pbar = tqdm(total=total_isrcs, desc="Fetching Data")

artists = []
release_dates = []

for isrc in new_features['isrc']:
    artist, release_date = fetch_artist_release_date(isrc)
    artists.append(artist)
    release_dates.append(release_date)
    pbar.update(1)  # Update progress

pbar.close()  # Close the progress bar

# сохраняем полученные данные в качестве новых признаков в исходном датасете
# Add 'artist' and 'release_date' columns to the 'new_features' dataset
new_features['artist'] = artists
new_features['release_date'] = release_dates

# сохраняем обновленный датасет (обновить путь)
new_features.to_csv('path_to_updated_df', index=False)



Fetching Data:   0%|                                                                             | 0/10585 [00:47<?, ?it/s]

Fetching Data: 100%|█████████████████████████████████████████████████████████████████| 10585/10585 [47:12<00:00,  3.74it/s]


OSError: Cannot save file into a non-existent directory: '/path/to/your'